In [1]:
# 스파크 세션 생성
import pyspark
from pyspark.sql import SparkSession

# SynapseML : 마이크로소프트에서 만든 스파크용 ML 라이브러리
# https://microsoft.github.io/SynapseML/
# config에서 synapseml 모듈 불러오기
spark = SparkSession.builder \
    .appName("Spark study - 231230") \
    .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:1.0.2") \
    .getOrCreate()

23/12/30 16:46:40 WARN Utils: Your hostname, KJH-DESKTOP resolves to a loopback address: 127.0.1.1; using 192.168.69.220 instead (on interface eth0)
23/12/30 16:46:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/kjh/app/spark/3.5.0/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/kjh/.ivy2/cache
The jars for the packages stored in: /home/kjh/.ivy2/jars
com.microsoft.azure#synapseml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c7ccd84a-a394-456d-b43e-54a5821491a8;1.0
	confs: [default]
	found com.microsoft.azure#synapseml_2.12;1.0.2 in central
	found com.microsoft.azure#synapseml-core_2.12;1.0.2 in central
	found org.apache.spark#spark-avro_2.12;3.4.1 in central
	found org.tukaani#xz;1.9 in central
	found commons-lang#commons-lang;2.6 in central
	found org.scalactic#scalactic_2.12;3.2.14 in central
	found org.scala-lang#scala-reflect;2.12.15 in central
	found io.spray#spray-json_2.12;1.3.5 in central
	found com.jcraft#jsch;0.1.54 in central
	found org.apache.httpcomponents.client5#httpclient5;5.1.3 in central
	found org.apache.httpcomponents.core5#httpcore5;5.1.3 in central
	found org.apache.httpcomponents.core5#httpcore5-h2;5.1.3 in central
	found org.slf4j#slf4j-api;1.7.25 in centra

In [2]:
df_movies = spark.read.parquet('file:///home/kjh/data/S.M.S/TMDB/*')
df_people = spark.read.parquet('file:///home/kjh/data/S.M.S/TMDB_people/')
df_genres = spark.read.parquet('file:///home/kjh/data/S.M.S/TMDB_genre/*')

In [3]:
df_movies.show(5)

+-------+--------------------+--------------------+--------------------+--------------------+---------------------+------+--------------------+--------+---------+-----------------+----------------------------------------+--------------------+--------------------+--------------------+--------------------+------------+-------+-------+---------------------+-----+-------+
|     id|                cast|                crew|             posters|       backdrop_path|belongs_to_collection|budget|              genres|homepage|  imdb_id|original_language|                          original_title|            overview|         poster_path|production_companies|production_countries|release_date|revenue|runtime|     spoken_languages|video|results|
+-------+--------------------+--------------------+--------------------+--------------------+---------------------+------+--------------------+--------+---------+-----------------+----------------------------------------+--------------------+----------------

In [4]:
df_people.show(5)

+-------+----------+---------------+--------------------+-----------+-----+-----+----+
|     id|  date_gte|           name|known_for_department|profile_img|birth|death|year|
+-------+----------+---------------+--------------------+-----------+-----+-----+----+
|1045960|2021-10-29|    Edward Kaye|              Acting|       NULL| NULL| NULL|2021|
|1083347|2021-10-29|    Lee Holland|              Acting|       NULL| NULL| NULL|2021|
| 115958|2021-10-29| Isabel Machado|           Directing|       NULL| NULL| NULL|2021|
|1160615|2021-10-29|Bruno Chouinard|           Directing|       NULL| NULL| NULL|2021|
|1186216|2021-10-29|   Petr Vachler|           Directing|       NULL| NULL| NULL|2021|
+-------+----------+---------------+--------------------+-----------+-----+-----+----+
only showing top 5 rows



In [5]:
df_genres.show(5)

+-------+------+---------+---------+------+-----+-----------+-----+------+-------+-------+------+-----+-------+-------+---------------+--------+--------+---+-------+
|     id|Action|Adventure|Animation|Comedy|Crime|Documentary|Drama|Family|Fantasy|History|Horror|Music|Mystery|Romance|Science_Fiction|TV_Movie|Thriller|War|Western|
+-------+------+---------+---------+------+-----+-----------+-----+------+-------+-------+------+-----+-------+-------+---------------+--------+--------+---+-------+
| 100028|     1|        0|        0|     0|    0|          0|    0|     0|      0|      0|     0|    0|      0|      0|              1|       1|       1|  0|      0|
|1000598|     0|        0|        0|     0|    0|          1|    0|     0|      0|      0|     0|    0|      0|      0|              0|       0|       0|  0|      0|
|1001013|     0|        0|        0|     0|    0|          0|    0|     0|      0|      0|     0|    0|      0|      0|              0|       0|       0|  0|      0|
|100

In [6]:
df_movies.createOrReplaceTempView('movies')
df_people.createOrReplaceTempView('people')
df_genres.createOrReplaceTempView('genres')

In [7]:
df_movies = spark.sql("""
                      SELECT id, original_title, release_date, original_language, runtime, array_union(cast, crew) AS people
                      FROM movies
                      """)

df_people = spark.sql("""
                      SELECT id, name, known_for_department
                      FROM people
                      """)

In [8]:
df_movies.show(5)
print(df_movies.count())

+-------+----------------------------------------+------------+-----------------+-------+--------------------+
|     id|                          original_title|release_date|original_language|runtime|              people|
+-------+----------------------------------------+------------+-----------------+-------+--------------------+
| 100028|                             Devil Winds|  2003-01-01|               en|     90|[18704, 44711, 46...|
|1000598|                    No Safe Place: Si...|  2003-01-01|               en|     50|[7090, 3271702, 3...|
|1001013|                         Brown Paper Bag|  2003-01-01|               en|     13|                  []|
|1001230|行け！レインボー仮面対ホームレス怪人軍団|  2003-01-01|               ja|     50|[1136684, 1104678...|
|1002578|                         Hunter-Gatherer|  2003-01-01|               en|     14|           [2656103]|
+-------+----------------------------------------+------------+-----------------+-------+--------------------+
only showing top 5 ro

633841


In [9]:
df_people.show(5)

+-------+---------------+--------------------+
|     id|           name|known_for_department|
+-------+---------------+--------------------+
|1045960|    Edward Kaye|              Acting|
|1083347|    Lee Holland|              Acting|
| 115958| Isabel Machado|           Directing|
|1160615|Bruno Chouinard|           Directing|
|1186216|   Petr Vachler|           Directing|
+-------+---------------+--------------------+
only showing top 5 rows



In [10]:
df_genres.show(5)
print(df_genres.count())

+-------+------+---------+---------+------+-----+-----------+-----+------+-------+-------+------+-----+-------+-------+---------------+--------+--------+---+-------+
|     id|Action|Adventure|Animation|Comedy|Crime|Documentary|Drama|Family|Fantasy|History|Horror|Music|Mystery|Romance|Science_Fiction|TV_Movie|Thriller|War|Western|
+-------+------+---------+---------+------+-----+-----------+-----+------+-------+-------+------+-----+-------+-------+---------------+--------+--------+---+-------+
| 100028|     1|        0|        0|     0|    0|          0|    0|     0|      0|      0|     0|    0|      0|      0|              1|       1|       1|  0|      0|
|1000598|     0|        0|        0|     0|    0|          1|    0|     0|      0|      0|     0|    0|      0|      0|              0|       0|       0|  0|      0|
|1001013|     0|        0|        0|     0|    0|          0|    0|     0|      0|      0|     0|    0|      0|      0|              0|       0|       0|  0|      0|
|100

633841


In [11]:
df_movies.createOrReplaceTempView('movies')
df_people.createOrReplaceTempView('people')
df_genres.createOrReplaceTempView('genres')

In [18]:
test = spark.sql("""
                 SELECT m.id, m.original_title, m.release_date, m.original_language, m.runtime, 
                    g.Action, g.Adventure, g.Animation, g.Comedy, g.Crime, g.Documentary, g.Drama, 
                    g.Family, g.Fantasy, g.History, g.Horror, g.Music, g.Mystery, 
                    g.Romance, g.Science_Fiction, g.TV_Movie, g.Thriller, g.War, g.Western,
                    explode(m.people) AS people
                FROM movies m
                INNER JOIN genres g
                ON m.id = g.id
                 """)

In [19]:
test.show()
print(test.count())

+---+--------------------+------------+-----------------+-------+------+---------+---------+------+-----+-----------+-----+------+-------+-------+------+-----+-------+-------+---------------+--------+--------+---+-------+------+
| id|      original_title|release_date|original_language|runtime|Action|Adventure|Animation|Comedy|Crime|Documentary|Drama|Family|Fantasy|History|Horror|Music|Mystery|Romance|Science_Fiction|TV_Movie|Thriller|War|Western|people|
+---+--------------------+------------+-----------------+-------+------+---------+---------+------+-----+-----------+-----+------+-------+-------+------+-----+-------+-------+---------------+--------+--------+---+-------+------+
| 22|Pirates of the Ca...|  2003-07-09|               en|    143|     1|        1|        0|     0|    0|          0|    0|     0|      1|      0|     0|    0|      0|      0|              0|       0|       0|  0|      0|    85|
| 22|Pirates of the Ca...|  2003-07-09|               en|    143|     1|        1|  

8081673


In [28]:
df = spark.sql("""
               SELECT temp.*, p.name AS people_name, p.known_for_department
               FROM (
                SELECT m.id, m.original_title, m.release_date, m.original_language, m.runtime, 
                    g.Action, g.Adventure, g.Animation, g.Comedy, g.Crime, g.Documentary, g.Drama, 
                    g.Family, g.Fantasy, g.History, g.Horror, g.Music, g.Mystery, 
                    g.Romance, g.Science_Fiction, g.TV_Movie, g.Thriller, g.War, g.Western,
                    explode(m.people) AS people
                FROM movies m
                INNER JOIN genres g
                ON m.id = g.id
               ) AS temp
               INNER JOIN people p
               ON temp.people = p.id
               ORDER BY temp.id
               """)

df = df.drop('people')

In [29]:
df.show()

+---+--------------+------------+-----------------+-------+------+---------+---------+------+-----+-----------+-----+------+-------+-------+------+-----+-------+-------+---------------+--------+--------+---+-------+-----------------+--------------------+
| id|original_title|release_date|original_language|runtime|Action|Adventure|Animation|Comedy|Crime|Documentary|Drama|Family|Fantasy|History|Horror|Music|Mystery|Romance|Science_Fiction|TV_Movie|Thriller|War|Western|      people_name|known_for_department|
+---+--------------+------------+-----------------+-------+------+---------+---------+------+-----+-----------+-----+------+-------+-------+------+-----+-------+-------+---------------+--------+--------+---+-------+-----------------+--------------------+
|  2|         Ariel|  1988-10-21|               fi|     73|     0|        0|        0|     1|    0|          0|    1|     0|      0|      0|     0|    0|      0|      1|              0|       0|       0|  0|      0| Susanna Haavisto|  